# Cost Calculation

**Objective:**

Estimate the production cost of a given part based on its reconstructed geometry. This includes calculating its physical length (in cm), estimating its mass (g) based on material density, and computing its cost using real-world price data.


#Load data

In [ ]:
import cv2
import pandas as pd
import numpy as np
import math

In [ ]:
url = "/content/materials_price.csv"

df = pd.read_csv(url)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   material         10 non-null     object
 1   density_g_cm3    10 non-null     object
 2   price_per_g_eur  10 non-null     object
 3   type             10 non-null     object
dtypes: object(4)
memory usage: 452.0+ bytes


In [ ]:
df.head()

,material,density_g_cm3,price_per_g_eur,type
0,PLA,1.24,0.030,plastic
1,ABS,1.04,0.030,plastic
2,PETG,1.27,0.035,plastic
3,Resin,1.10,0.050,resin
4,Silver,10.49,1.140,metal


Return density and price for a given material

In [ ]:
cols = ["material", "density_g_cm3", "price_per_g_eur"]
num_cols = ["density_g_cm3", "price_per_g_eur"]

In [ ]:
for col in cols:
  df[col] = df[col].astype(str)
  df[col] = df[col].str.replace(",", ".", regex=False).str.strip()

In [ ]:
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors= 'coerce')

# Get materials properties

In [ ]:
def get_material_props(material):
    """
    Return density (g/cm3) and price (€/g) for given material.
    """
    row = df[df.material.str.lower() == material.lower()]
    if row.empty:
        raise ValueError(f"{material} not found in database.")

    density = float(row.density_g_cm3)
    price_per_g = float(row.price_per_g_eur)

    return density, price_per_g

In [ ]:
density, price = get_material_props("Gold")
print(f"Density: {density} g/cm3, Price: €{price}/g")

density, price = get_material_props("PLA")
print(f"Density: {density} g/cm3, Price: €{price}/g")

Density: 19.32 g/cm3, Price: €99.26/g
Density: 1.24 g/cm3, Price: €0.03/g


/tmp/ipython-input-96-4225156941.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  density = float(row.density_g_cm3)
/tmp/ipython-input-96-4225156941.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price_per_g = float(row.price_per_g_eur)


# Ring size conversion

In [ ]:
ring_sizes = {
    "EU": {
        45: 14.3, 46:14.7, 47:15.1, 48:15.5, 49:15.7, 52:16.7, 55:17.5, 57:18.2, 60:18.9, 62:19.8
        },
    "US": {
        5: 15.7, 6: 16.7, 7: 17.5, 8: 18.1, 9: 18.9, 10: 19.8
    }
}

In [ ]:
def get_ring_diameter(size, region = "US"):
    """Return ring inner diameter in mm"""
    return ring_sizes.get(region, {}).get(size, None)

In [ ]:
get_ring_diameter(7)

17.3

# Estimate volume and mass


In [ ]:
def estimate_ring_band_volume(inner_diameter_mm, thickness_mm = 2.0):
    """
    Approximate volume of the ring band (argolla) as a torus in cm^3
    """
    inner_radius_cm = inner_diameter_mm / 2 / 10
    tube_radius_cm = thickness_mm / 2 / 10
    volume = 2 * math.pi**2 * inner_radius_cm * tube_radius_cm**2
    return volume

# Volume from skeleton

In [ ]:
#we know our skeleton have a length = 5023 pixels

def pixels_to_cm(pixels: float, reference_cm= 10.0, reference_pixels = 255):
    return round(pixels * (reference_cm / reference_pixels), 4)

In [ ]:
pixels_to_cm(5023, 10)

196.9804

In [ ]:
def calculate_volume_from_skeleton(length_cm, thickness_mm):
    thickness_cm = thickness_mm / 10
    return round(length_cm * (thickness_cm ** 2), 4)

In [ ]:
calculate_volume_from_skeleton(pixels_to_cm(5023, 5))

3.9396

# Cost estimation

In [ ]:
def estimate_accessory_cost(
    material,
    skeleton_length_cm,
    accessory_type,
    include_band= False,
    ring_size = None,
    thickness_mm= 2.0
):
    density, price_per_g = get_material_props(material)

    # Volumen del escudo/esqueleto
    skeleton_volume = calculate_volume_from_skeleton(skeleton_length_cm, thickness_mm)

    # Volumen adicional si es anillo
    band_volume = 0
    if accessory_type == "ring" and include_band:
        if ring_size is None:
            raise ValueError("Ring size must be specified if include_band=True for a ring.")
        diameter_mm = get_ring_diameter(ring_size)
        if diameter_mm is None:
            raise ValueError("Invalid ring size.")
        band_volume = estimate_ring_band_volume(diameter_mm, thickness_mm)

    total_volume = skeleton_volume + band_volume
    total_mass = total_volume * density
    total_cost = total_mass * price_per_g

    return {
        "volume_cm3": round(total_volume, 4),
        "mass_g": round(total_mass, 4),
        "cost_eur": round(total_cost, 4)
    }

In [ ]:
estimate_accessory_cost(
    material="steel",
    skeleton_length_cm=98.4902,
    accessory_type="ring",
    include_band=True,
    ring_size=8,
    thickness_mm=2.0
)

/tmp/ipython-input-96-4225156941.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  density = float(row.density_g_cm3)
/tmp/ipython-input-96-4225156941.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price_per_g = float(row.price_per_g_eur)


{'volume_cm3': 4.1182, 'mass_g': 32.1223, 'cost_eur': 0.0282}

In [ ]:
estimate_accessory_cost(
    material="PLA",
    skeleton_length_cm=98.4902,
    accessory_type="ring",
    include_band=True,
    ring_size=8,
    thickness_mm=2.0
)

/tmp/ipython-input-96-4225156941.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  density = float(row.density_g_cm3)
/tmp/ipython-input-96-4225156941.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price_per_g = float(row.price_per_g_eur)


{'volume_cm3': 4.1182, 'mass_g': 5.1066, 'cost_eur': 0.1532}